In [ ]:
import os
import shutil
import subprocess
import tempfile
import requests
from bs4 import BeautifulSoup
from clearml import PipelineDecorator
from pymongo import MongoClient
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound

# Simulating logger for simplicity
class Logger:
    @staticmethod
    def info(message):
        print(f"[INFO] {message}")

logger = Logger()

# Mock of RepositoryDocument to simulate database actions
class Document:
    # Connect to the ClearML MongoDB server
    client = MongoClient("mongodb://localhost:27017/")  # Adjust connection string if necessary
    db = client['clearml']  # Replace 'clearml' with your database name if different
    collection = db['crawled_data']  # Specify the collection name

    @staticmethod
    def find(link):
        # Check if the document with the given link already exists
        return Document.collection.find_one({"link": link})

    def __init__(self, content, name, link, platform):
        self.content = content
        self.name = name
        self.link = link
        self.platform = platform

    def save(self):
        # Insert the document into the MongoDB collection
        document = {
            "content": self.content,
            "name": self.name,
            "link": self.link,
            "platform": self.platform,
        }
        result = Document.collection.insert_one(document)
        print(f"Document saved with ID: {result.inserted_id}")


# BaseCrawler class
class BaseCrawler:
    def __init__(self):
        pass

    def extract(self, link: str) -> None:
        raise NotImplementedError("Extract method must be implemented by subclasses.")

class YouTubeCrawler(BaseCrawler):
    model = Document

    def extract(self, link: str) -> None:
        old_model = self.model.find(link=link)
        if old_model is not None:
            logger.info(f"YouTube video already exists in the database: {link}")
            return

        logger.info(f"Starting scraping YouTube video: {link}")

        try:
            # Extract video ID from the link
            video_id = link.split("v=")[-1].split("&")[0]

            # Fetch transcript
            try:
                transcript = YouTubeTranscriptApi.get_transcript(video_id)
                transcript_text = " ".join([entry["text"] for entry in transcript])
            except (TranscriptsDisabled, NoTranscriptFound):
                transcript_text = "Transcript not available for this video."

            # Fetch video title
            response = requests.get(link)
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.find('title').text.strip()

            instance = self.model(
                content=transcript_text,
                name=title,
                link=link,
                platform="youtube",
            )
            instance.save()

        except Exception as e:
            print(f"Error during extraction: {e}")

        logger.info(f"Finished scraping YouTube video: {link}")
# GitHub Crawler
class GithubCrawler(BaseCrawler):
    model = Document

    def __init__(self, ignore=(".git", ".toml", ".lock", ".png")) -> None:
        super().__init__()
        self._ignore = ignore

    def extract(self, link: str) -> None:
        old_model = self.model.find(link=link)
        if old_model is not None:
            logger.info(f"Repository already exists in the database: {link}")
            return

        logger.info(f"Starting scraping GitHub repository: {link}")

        repo_name = link.rstrip("/").split("/")[-1]
        local_temp = tempfile.mkdtemp()

        try:
            os.chdir(local_temp)
            subprocess.run(["git", "clone", link], check=True)

            repo_path = os.path.join(local_temp, os.listdir(local_temp)[0])

            tree = {}
            for root, _, files in os.walk(repo_path):
                dir = root.replace(repo_path, "").lstrip("/")
                if any(dir.startswith(ignore_item) for ignore_item in self._ignore):
                    continue

                for file in files:
                    if any(file.endswith(ignore_item) for ignore_item in self._ignore):
                        continue
                    file_path = os.path.join(dir, file)
                    with open(os.path.join(root, file), "r", errors="ignore") as f:
                        tree[file_path] = f.read().replace(" ", "")

            instance = self.model(
                content=tree,
                name=repo_name,
                link=link,
                platform="github",
            )
            instance.save()

        except Exception as e:
            print(f"Error during extraction: {e}")
        finally:
            shutil.rmtree(local_temp)

        logger.info(f"Finished scraping GitHub repository: {link}")

# LinkedIn Crawler
class LinkedInCrawler(BaseCrawler):
    model = Document

    def extract(self, link: str) -> None:
        old_model = self.model.find(link=link)
        if old_model is not None:
            logger.info(f"LinkedIn profile already exists in the database: {link}")
            return

        logger.info(f"Starting scraping LinkedIn profile: {link}")

        try:
            response = requests.get(link)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Simplified example: fetch profile name and headline
            profile_name = soup.find('title').text.strip()
            content = soup.prettify()

            instance = self.model(
                content=content,
                name=profile_name,
                link=link,
                platform="linkedin",
            )
            instance.save()

        except Exception as e:
            print(f"Error during extraction: {e}")

        logger.info(f"Finished scraping LinkedIn profile: {link}")

# Medium Crawler
class MediumCrawler(BaseCrawler):
    model = Document

    def extract(self, link: str) -> None:
        old_model = self.model.find(link=link)
        if old_model is not None:
            logger.info(f"Medium article already exists in the database: {link}")
            return

        logger.info(f"Starting scraping Medium article: {link}")

        try:
            response = requests.get(link)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Simplified example: fetch article title and content
            article_title = soup.find('h1').text.strip()
            article_body = " ".join([p.text for p in soup.find_all('p')])

            instance = self.model(
                content=article_body,
                name=article_title,
                link=link,
                platform="medium",
            )
            instance.save()

        except Exception as e:
            print(f"Error during extraction: {e}")

        logger.info(f"Finished scraping Medium article: {link}")

# Custom Article Crawler
class CustomCrawler(BaseCrawler):
    model = Document

    def extract(self, link: str) -> None:
        old_model = self.model.find(link=link)
        if old_model is not None:
            logger.info(f"Custom article already exists in the database: {link}")
            return

        logger.info(f"Starting scraping custom article: {link}")

        try:
            response = requests.get(link)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Fetch title and basic content
            title = soup.find('title').text.strip()
            body = " ".join([p.text for p in soup.find_all('p')])

            instance = self.model(
                content=body,
                name=title,
                link=link,
                platform="custom",
            )
            instance.save()

        except Exception as e:
            print(f"Error during extraction: {e}")

        logger.info(f"Finished scraping custom article: {link}")

# Usage example
def test_crawlers():
    crawlers = {
        "youtube": YouTubeCrawler(),

    }

    links = [
        ("youtube", "https://www.youtube.com/watch?v=dQw4w9WgXcQ"),
    ]

    for platform, link in links:
        crawler = crawlers.get(platform)
        if crawler:
            crawler.extract(link)

test_crawlers()


[INFO] Starting scraping YouTube video: https://www.youtube.com/watch?v=dQw4w9WgXcQ
Document saved with ID: 6753eec62758826212b3342e
[INFO] Finished scraping YouTube video: https://www.youtube.com/watch?v=dQw4w9WgXcQ
